# スタッキング002

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [7]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 238)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [8]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [9]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [11]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1172 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1126 features are remained after removing non-informative features
129 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 997 features are remained after removing features not interesting for LightGBM classifier
---=> 54 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1051 features

---=> df final shape: (356250, 1051)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [14]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [15]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356250, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [16]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,1,0,1,1,0,0,0,0,0
1,307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,0,1,0,1,1,0,1,0
4,307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


### modelを生成

In [19]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [20]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [21]:
# infをnanへ置換
app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [22]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [23]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [24]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,307506,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,1,1,0,0,0,0,0
100005,307507,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,1,0,0,0,0
100013,307508,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,0,1,0,1,1,0,1,0
100028,307509,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,0,1,1,0,1,0
100038,307510,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,1,0,0,0,0,0


In [25]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,-0.232482,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,-0.907160,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,0.217303,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,2.578675,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,0.206059,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [26]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
1,100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
2,100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
3,100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
4,100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [27]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100002,-1.732422,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.165405,...,0.0,0.579590,-0.010361,0.580078,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0
1,100003,-1.732422,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.585938,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
2,100004,-1.732422,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.391602,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
3,100006,-1.732422,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.175171,...,0.0,-1.724609,-0.010361,-1.723633,-1.056641,1.190430,1.684570,-0.150513,-0.498047,0.0
4,100007,-1.732422,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.358887,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0


In [28]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100001,1.732422,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075134,-0.447998,...,0.0,0.579590,-0.010361,0.580078,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0
1,100005,1.732422,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.666016,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
2,100013,1.732422,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.912109,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0
3,100028,1.732422,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.495117,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0
4,100038,1.732422,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.337646,...,0.0,-1.724609,-0.010361,-1.723633,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0


In [29]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [30]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [31]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Lgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[1]	train's auc: 0.736875	valid's auc: 0.724037
[2]	train's auc: 0.74117	valid's auc: 0.727936
[3]	train's auc: 0.746596	valid's auc: 0.732995
[4]	train's auc: 0.749215	valid's auc: 0.735807
[5]	train's auc: 0.751124	valid's auc: 0.737459
[6]	train's auc: 0.751605	valid's auc: 0.737185
[7]	train's auc: 0.752542	valid's auc: 0.738328
[8]	train's auc: 0.752998	valid's auc: 0.738784
[9]	train's auc: 0.753219	valid's auc: 0.738672
[10]	train's auc: 0.753414	valid's auc: 0.73874
[11]	train's auc: 0.753789	valid's auc: 0.738832
[12]	train's auc: 0.754367	valid's auc: 0.7394
[13]	train's auc: 0.754507	valid's auc: 0.739578
[14]	train's auc: 0.754608	valid's auc: 0.739395
[15]	train's auc: 0.754733	valid's auc: 0.739591
[16]	train's auc: 0.755108	valid's auc: 0.73984
[17]	train's auc: 0.75544	valid's auc: 0.740236
[18]	train's auc: 0.755478	valid's auc: 0.740304
[19]	train's auc: 0.755771	val

[166]	train's auc: 0.784326	valid's auc: 0.759819
[167]	train's auc: 0.784511	valid's auc: 0.759921
[168]	train's auc: 0.784698	valid's auc: 0.760028
[169]	train's auc: 0.784872	valid's auc: 0.760139
[170]	train's auc: 0.78505	valid's auc: 0.760248
[171]	train's auc: 0.785259	valid's auc: 0.760383
[172]	train's auc: 0.785422	valid's auc: 0.76051
[173]	train's auc: 0.785588	valid's auc: 0.760601
[174]	train's auc: 0.785763	valid's auc: 0.760709
[175]	train's auc: 0.785938	valid's auc: 0.760804
[176]	train's auc: 0.786092	valid's auc: 0.760882
[177]	train's auc: 0.786301	valid's auc: 0.761023
[178]	train's auc: 0.786438	valid's auc: 0.761111
[179]	train's auc: 0.786582	valid's auc: 0.761204
[180]	train's auc: 0.786729	valid's auc: 0.761312
[181]	train's auc: 0.786887	valid's auc: 0.761425
[182]	train's auc: 0.787054	valid's auc: 0.761536
[183]	train's auc: 0.787196	valid's auc: 0.76164
[184]	train's auc: 0.787355	valid's auc: 0.761755
[185]	train's auc: 0.787517	valid's auc: 0.761857
[18

[332]	train's auc: 0.808705	valid's auc: 0.775063
[333]	train's auc: 0.808854	valid's auc: 0.77515
[334]	train's auc: 0.808961	valid's auc: 0.775204
[335]	train's auc: 0.809086	valid's auc: 0.775261
[336]	train's auc: 0.809206	valid's auc: 0.775315
[337]	train's auc: 0.809309	valid's auc: 0.775364
[338]	train's auc: 0.809429	valid's auc: 0.775418
[339]	train's auc: 0.809566	valid's auc: 0.775523
[340]	train's auc: 0.809688	valid's auc: 0.775587
[341]	train's auc: 0.809796	valid's auc: 0.775636
[342]	train's auc: 0.809909	valid's auc: 0.775688
[343]	train's auc: 0.810031	valid's auc: 0.775769
[344]	train's auc: 0.810154	valid's auc: 0.775827
[345]	train's auc: 0.810265	valid's auc: 0.775883
[346]	train's auc: 0.81038	valid's auc: 0.775943
[347]	train's auc: 0.810485	valid's auc: 0.775989
[348]	train's auc: 0.810598	valid's auc: 0.776063
[349]	train's auc: 0.810699	valid's auc: 0.776119
[350]	train's auc: 0.810805	valid's auc: 0.7762
[351]	train's auc: 0.810923	valid's auc: 0.776248
[352

[497]	train's auc: 0.825776	valid's auc: 0.782783
[498]	train's auc: 0.825856	valid's auc: 0.782827
[499]	train's auc: 0.825946	valid's auc: 0.782848
[500]	train's auc: 0.82604	valid's auc: 0.782894
[501]	train's auc: 0.826129	valid's auc: 0.782936
[502]	train's auc: 0.826223	valid's auc: 0.782954
[503]	train's auc: 0.826308	valid's auc: 0.782982
[504]	train's auc: 0.826391	valid's auc: 0.783016
[505]	train's auc: 0.826468	valid's auc: 0.78304
[506]	train's auc: 0.826564	valid's auc: 0.783072
[507]	train's auc: 0.826638	valid's auc: 0.783096
[508]	train's auc: 0.826737	valid's auc: 0.783143
[509]	train's auc: 0.82683	valid's auc: 0.783191
[510]	train's auc: 0.826917	valid's auc: 0.783229
[511]	train's auc: 0.826998	valid's auc: 0.783257
[512]	train's auc: 0.82709	valid's auc: 0.783299
[513]	train's auc: 0.827185	valid's auc: 0.783325
[514]	train's auc: 0.827257	valid's auc: 0.783352
[515]	train's auc: 0.827344	valid's auc: 0.783376
[516]	train's auc: 0.827428	valid's auc: 0.783407
[517

[663]	train's auc: 0.839356	valid's auc: 0.787088
[664]	train's auc: 0.839432	valid's auc: 0.787109
[665]	train's auc: 0.839507	valid's auc: 0.787124
[666]	train's auc: 0.839576	valid's auc: 0.787123
[667]	train's auc: 0.839642	valid's auc: 0.787149
[668]	train's auc: 0.839728	valid's auc: 0.78717
[669]	train's auc: 0.839797	valid's auc: 0.787194
[670]	train's auc: 0.839881	valid's auc: 0.787237
[671]	train's auc: 0.839956	valid's auc: 0.787251
[672]	train's auc: 0.840028	valid's auc: 0.787267
[673]	train's auc: 0.840104	valid's auc: 0.787281
[674]	train's auc: 0.840179	valid's auc: 0.787301
[675]	train's auc: 0.840251	valid's auc: 0.787329
[676]	train's auc: 0.840324	valid's auc: 0.787348
[677]	train's auc: 0.840393	valid's auc: 0.787363
[678]	train's auc: 0.840466	valid's auc: 0.78739
[679]	train's auc: 0.840543	valid's auc: 0.787406
[680]	train's auc: 0.840625	valid's auc: 0.787417
[681]	train's auc: 0.840699	valid's auc: 0.787446
[682]	train's auc: 0.840759	valid's auc: 0.787476
[6

...

[4241]	train's auc: 0.965737	valid's auc: 0.793676
[4242]	train's auc: 0.965753	valid's auc: 0.793675
[4243]	train's auc: 0.965772	valid's auc: 0.793673
[4244]	train's auc: 0.965788	valid's auc: 0.79367
[4245]	train's auc: 0.965803	valid's auc: 0.79367
[4246]	train's auc: 0.965817	valid's auc: 0.793665
[4247]	train's auc: 0.96583	valid's auc: 0.79366
[4248]	train's auc: 0.965846	valid's auc: 0.793661
[4249]	train's auc: 0.965862	valid's auc: 0.793663
[4250]	train's auc: 0.96588	valid's auc: 0.793666
[4251]	train's auc: 0.96589	valid's auc: 0.793663
[4252]	train's auc: 0.965907	valid's auc: 0.793664
[4253]	train's auc: 0.965923	valid's auc: 0.793661
[4254]	train's auc: 0.965935	valid's auc: 0.793664
[4255]	train's auc: 0.965947	valid's auc: 0.793661
[4256]	train's auc: 0.965958	valid's auc: 0.79367
[4257]	train's auc: 0.965971	valid's auc: 0.793673
[4258]	train's auc: 0.965988	valid's auc: 0.793678
[4259]	train's auc: 0.966006	valid's auc: 0.793674
[4260]	train's auc: 0.966022	valid's a

[4403]	train's auc: 0.968137	valid's auc: 0.79355
[4404]	train's auc: 0.968147	valid's auc: 0.79355
[4405]	train's auc: 0.968162	valid's auc: 0.793552
[4406]	train's auc: 0.968178	valid's auc: 0.793554
[4407]	train's auc: 0.968193	valid's auc: 0.79355
[4408]	train's auc: 0.968209	valid's auc: 0.79355
[4409]	train's auc: 0.968228	valid's auc: 0.79354
[4410]	train's auc: 0.96824	valid's auc: 0.793537
[4411]	train's auc: 0.968252	valid's auc: 0.793539
[4412]	train's auc: 0.968269	valid's auc: 0.793535
[4413]	train's auc: 0.968285	valid's auc: 0.793534
[4414]	train's auc: 0.968295	valid's auc: 0.793532
[4415]	train's auc: 0.968312	valid's auc: 0.793531
[4416]	train's auc: 0.96833	valid's auc: 0.793528
[4417]	train's auc: 0.968343	valid's auc: 0.79353
[4418]	train's auc: 0.968362	valid's auc: 0.79353
[4419]	train's auc: 0.968374	valid's auc: 0.793536
[4420]	train's auc: 0.968382	valid's auc: 0.793533
[4421]	train's auc: 0.968391	valid's auc: 0.793533
[4422]	train's auc: 0.968406	valid's auc

[4566]	train's auc: 0.970512	valid's auc: 0.793417
[4567]	train's auc: 0.970522	valid's auc: 0.79342
[4568]	train's auc: 0.970539	valid's auc: 0.793422
[4569]	train's auc: 0.970559	valid's auc: 0.79342
[4570]	train's auc: 0.970573	valid's auc: 0.793424
[4571]	train's auc: 0.970589	valid's auc: 0.793422
[4572]	train's auc: 0.9706	valid's auc: 0.793422
[4573]	train's auc: 0.97062	valid's auc: 0.793427
[4574]	train's auc: 0.970628	valid's auc: 0.793424
[4575]	train's auc: 0.970644	valid's auc: 0.793423
[4576]	train's auc: 0.970659	valid's auc: 0.793422
[4577]	train's auc: 0.970681	valid's auc: 0.793423
[4578]	train's auc: 0.970692	valid's auc: 0.793429
[4579]	train's auc: 0.970706	valid's auc: 0.793425
[4580]	train's auc: 0.970717	valid's auc: 0.793418
[4581]	train's auc: 0.970738	valid's auc: 0.793418
[4582]	train's auc: 0.970748	valid's auc: 0.79342
[4583]	train's auc: 0.970759	valid's auc: 0.793422
[4584]	train's auc: 0.97077	valid's auc: 0.793422
[4585]	train's auc: 0.970777	valid's a

[4728]	train's auc: 0.972728	valid's auc: 0.793367
[4729]	train's auc: 0.972736	valid's auc: 0.793363
[4730]	train's auc: 0.972747	valid's auc: 0.793365
[4731]	train's auc: 0.972759	valid's auc: 0.793359
[4732]	train's auc: 0.972767	valid's auc: 0.793355
[4733]	train's auc: 0.972779	valid's auc: 0.793356
[4734]	train's auc: 0.972787	valid's auc: 0.793355
[4735]	train's auc: 0.972802	valid's auc: 0.793338
[4736]	train's auc: 0.972813	valid's auc: 0.79334
[4737]	train's auc: 0.972823	valid's auc: 0.793337
[4738]	train's auc: 0.972834	valid's auc: 0.793338
[4739]	train's auc: 0.972843	valid's auc: 0.793337
[4740]	train's auc: 0.972854	valid's auc: 0.79334
[4741]	train's auc: 0.972862	valid's auc: 0.793339
[4742]	train's auc: 0.972875	valid's auc: 0.793341
[4743]	train's auc: 0.972888	valid's auc: 0.793335
[4744]	train's auc: 0.972911	valid's auc: 0.793332
[4745]	train's auc: 0.972924	valid's auc: 0.793332
[4746]	train's auc: 0.972934	valid's auc: 0.793332
[4747]	train's auc: 0.972949	vali

[4891]	train's auc: 0.974797	valid's auc: 0.793233
[4892]	train's auc: 0.974807	valid's auc: 0.793234
[4893]	train's auc: 0.974822	valid's auc: 0.793231
[4894]	train's auc: 0.974835	valid's auc: 0.793232
[4895]	train's auc: 0.974849	valid's auc: 0.793233
[4896]	train's auc: 0.974863	valid's auc: 0.793224
[4897]	train's auc: 0.97487	valid's auc: 0.793219
[4898]	train's auc: 0.974886	valid's auc: 0.793217
[4899]	train's auc: 0.974906	valid's auc: 0.793215
[4900]	train's auc: 0.974917	valid's auc: 0.793212
[4901]	train's auc: 0.974927	valid's auc: 0.79321
[4902]	train's auc: 0.974939	valid's auc: 0.793213
[4903]	train's auc: 0.974959	valid's auc: 0.793215
[4904]	train's auc: 0.974977	valid's auc: 0.79322
[4905]	train's auc: 0.974993	valid's auc: 0.793222
[4906]	train's auc: 0.975004	valid's auc: 0.793227
[4907]	train's auc: 0.975013	valid's auc: 0.793231
[4908]	train's auc: 0.975023	valid's auc: 0.793225
[4909]	train's auc: 0.975034	valid's auc: 0.793223
[4910]	train's auc: 0.975052	valid

In [36]:
pred_test_1b

array([0.02502306, 0.09033532, 0.03703325, ..., 0.00713924, 0.02038091,
       0.25704715])

In [37]:
pred_train_1b

array([0.15604847, 0.01203393, 0.02633327, ..., 0.03491565, 0.11710027,
       0.05404276])

In [39]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [40]:
print(type(pred_1b))

<class 'list'>


In [41]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2a': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set2a
0,100002,0.156048
1,100003,0.012034
2,100004,0.026333
3,100006,0.029856
4,100007,0.034698


In [42]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set2a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [43]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [44]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2a': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [45]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set2a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [46]:
del df
del x_train
del y_train
del x_test
del app_train
del app_test
del model_1b
gc.collect()

2

## 特徴量生成2